<a href="https://colab.research.google.com/github/MarcosP7635/power_densities/blob/main/Plot_power.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This is a [Google Colab](https://colab.research.google.com/) [Jupyter notebook](https://jupyter.org/). To run the Python code in this notebook, scroll down then click each play button that appears below. This only needs to be done once per play button.

#Load Data. Please press the play button. It will take about 1 minute to execute. 

In [1]:
import pandas as pd
import numpy as np
!pip install -U plotly
import plotly.express as px
import ipywidgets #as widgets
!wget https://github.com/MarcosP7635/power_densities/raw/main/Corrected_power_time_series_201_steps.csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 5.7 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
--2022-10-18 03:12:13--  https://github.com/MarcosP7635/power_densities/raw/main/Corrected_power_time_series_201_steps.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MarcosP7635/power_densities/main/Corrected_power_time_series_201_steps.csv [following]
--2022-10-18 03:12:13--  https://raw.githubusercontent.com/MarcosP7635/power_densities/main/Corrected_power_time_series_201_steps.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting 

Download the data from a public Github repo

In [2]:
high_res_time_series_df = pd.read_csv(
    "/content/Corrected_power_time_series_201_steps.csv",
    on_bad_lines = 'warn')

Create the dataframe that will be used for plotting

In [3]:
high_res_time_series_df.set_index('Parent Isotope', inplace = True)
time_array = [float(t) for t in high_res_time_series_df.columns]

The functions used to create plots. 

In [32]:
def get_max(entries):
  cols = [''.join([l for l in entry if l.isdigit()]) + 
          ''.join([l for l in entry if l.isalpha()])
          for entry in entries.split(",")]
  max_y = np.max(high_res_time_series_df.loc[cols[0]])
  for col in cols[1:]:
    max_y = max(max_y, np.max(high_res_time_series_df.loc[col]))
  return max_y
import plotly.express as px
import plotly.graph_objects as go
def plot(entries):
  fig = go.Figure()
  if "," in entries:
    title = "Power Produced by 1 Gram Each of "
  else:
    title = "Power Produced by 1 Gram of "
  for entry in entries.split(","):
    A = int(''.join([l for l in entry if l.isdigit()]))
    El = ''.join([l for l in entry if l.isalpha()])
    entry = str(A) + El
    if np.max(high_res_time_series_df.loc[entry]) < 10**-6 :
      fig.update_layout(title = entry + " produces less than 1μW") 
      return fig
    else:
      fig.add_trace(go.Scatter(x = time_array,
            y = high_res_time_series_df.loc[entry] / A,
            name = El + "-" + str(A), mode = "markers"))
      y_dict = dict(type = "log", title = "Power (Watts)", nticks = 16,
              range = [-6, np.log10(get_max(entries))] )
      x_dict = dict(type = "log", title = "Time (Seconds)", nticks = 24,
                    range = [-10, 10])
      fig.update_layout(xaxis = x_dict, yaxis = y_dict,
              title = title + entries)  
  return fig

Create all of the widgets to make the plotting interactive.

In [5]:
from IPython.display import display, clear_output
button = ipywidgets.Button(description='Make Plot')
button_multiple = ipywidgets.Button(description='Make Plot')
output = ipywidgets.Output()
dropdown = ipywidgets.Dropdown(
            options=[x for x in high_res_time_series_df.index],
            value = "3H")
text_box = ipywidgets.Text(description="Type here")

def on_button_clicked_1_isotope(b):
  with output:
    clear_output()
    plot(dropdown.value).show()

button.on_click(on_button_clicked_1_isotope)

def on_button_clicked_multiple_isotopes(c):
  with output:
    clear_output()
    plot(text_box.value).show()   

button_multiple.on_click(on_button_clicked_multiple_isotopes)

#Plot Data Interactively


After clicking on the dropdown box, you can type the name of an isotope to find it in the list. Click "Make Plot" once you have selected an isotope. 

In [6]:
display(dropdown, button, output)

Dropdown(index=773, options=('113Sn', '243Cf', '168Eu', '162Sm', '57V', '208Po', '174Ir', '144Gd', '80Sr', '33…

Button(description='Make Plot', style=ButtonStyle())

Output()

#Plot and compare multiple isotopes

Enter the isotope(s) to plot, each isotope separated by a comma. Click "Make Plot" once you have entered the isotope(s). 

In [7]:
display(text_box, button_multiple, output)

Text(value='', description='Type here')

Button(description='Make Plot', style=ButtonStyle())

Output()